In [38]:
import os
# import torch
# from datasets import load_dataset, Dataset
import pandas as pd
# from transformers import (
#     AutoModelForCausalLM,
#     AutoModel,
#     AutoTokenizer,
#     BitsAndBytesConfig,
#     HfArgumentParser,
#     TrainingArguments,
#     pipeline,
#     logging,
# )

# from sentence_transformers import SentenceTransformer, util
from rank_bm25 import BM25Okapi
import random
import numpy as np
from typing import Literal, Optional, TypedDict
import json

In [39]:
ICL_METHOD = "random"

In [40]:
def read_split_ids(ids_path:str) -> list[int]:
    with open(ids_path, 'r', encoding='utf-8') as ids_file:
        ids_lst = ids_file.readlines()
    ids_lst = [int(id) for id in ids_lst]
    return ids_lst

In [41]:
dev_ids = read_split_ids('../data/few_data_split/dev_ids.txt')
train_ids = read_split_ids('../data/few_data_split/dev_ids.txt')

In [42]:
with open('../data/few_data_split/input_data.json', 'r', encoding='utf-8') as json_file:
    input_data = json.load(json_file)
print(input_data[:5])

[{'id': 0, 'name': 'UUParser', 'fullname': 'N/A', 'description': ['a near-SOTA model', 'a variant of the K&G transition-based parser that employs the arc-hybrid transition system from Kuhlmann et al. (2011) extended with a S WAP transition and a Static-Dynamic oracle'], 'citationtag': ['de Lhoneux et al. (2017b)'], 'role': 'Method', 'type': 'Code', 'func': 'Use', 'url': 'https://github.com/mdelhoneux/uuparser-composition', 'section_title': '4 Composition in a K&G Parser', 'add_info': '4 The code can be found at https://github.com/mdelhoneux/uuparser-composition', 'text': 'Parser We use UUParser, a variant of the K&G transition-based parser that employs the arc-hybrid transition system from Kuhlmann et al. (2011) extended with a S WAP transition and a Static-Dynamic oracle, as described in de Lhoneux et al. (2017b) [Cite_Footnote_4] . The S WAP transition is used to allow the construction of non-projective dependency trees (Nivre, 2009). We use default hyperparameters. When using POS ta

In [43]:
def create_icl(input_data:list[dict], train_ids:list[int], dev_ids:list[int], method:str, k:int=5) -> list[list[str]]:
    icl_idxs = []
    dev_data = [input_data[dev_id] for dev_id in dev_ids]
    train_data = [input_data[train_id] for train_id in train_ids]

    # random
    if method == "random":
        for i, cont in enumerate(dev_data):
            random.seed(i)
            icl_idxs.append(random.sample(train_ids, k))

    #bm25
    elif method == "bm25":
        tokenized_corpus = [cont['text'].split(" ") for cont in train_data]
        bm25 = BM25Okapi(tokenized_corpus)

        for i, cont in enumerate(dev_data):
            bm25_scores = bm25.get_scores(cont['text'].split(" "))
            reset_ids = np.argsort(bm25_scores)[-k:][::-1].tolist()
            icl_idxs.append([train_data[reset_id]['id'] for reset_id in reset_ids])
    # elif method == "encoder":
    #     model = SentenceTransformer("intfloat/multilingual-e5-base")

    #     tokenized_corpus = model.encode(train_cont_3sent, convert_to_tensor=True)
    #     for cont, (i, row) in zip(test_cont_3sent, test_df.iterrows()):
    #         tokenized_query = model.encode(test_cont_3sent)
    #         cos_scores = util.cos_sim(tokenized_query, tokenized_corpus)[0]

    #         icl_idxs.append(np.argsort(-cos_scores)[:k].tolist())
    else:
        print("select other method")
    
    return icl_idxs

In [44]:
icls = create_icl(input_data=input_data, train_ids=train_ids, dev_ids=dev_ids, method=ICL_METHOD)
print(icls)

[[123, 335, 395, 220, 187], [297, 413, 349, 64, 62], [12, 118, 242, 308, 331], [391, 195, 175, 61, 141], [391, 282, 18, 409, 318], [64, 121, 285, 225, 139], [72, 242, 261, 220, 396], [85, 90, 409, 13, 176], [293, 141, 423, 61, 147], [139, 141, 185, 297, 320], [72, 396, 286, 318, 231], [332, 170, 139, 187, 195], [82, 185, 285, 112, 279], [220, 81, 320, 293, 279], [18, 285, 47, 185, 64], [137, 231, 298, 396, 180], [308, 82, 318, 420, 335], [298, 335, 282, 308, 81], [320, 62, 332, 433, 391], [395, 298, 62, 187, 196], [90, 220, 166, 85, 72], [331, 335, 420, 318, 68], [297, 47, 225, 332, 320], [81, 242, 5, 195, 370], [123, 321, 320, 68, 331], [423, 231, 68, 370, 82], [196, 137, 312, 316, 175], [318, 350, 420, 196, 176], [143, 61, 175, 316, 429], [323, 176, 112, 316, 117], [175, 81, 225, 137, 64], [231, 82, 143, 409, 279], [176, 68, 279, 282, 391], [72, 331, 293, 350, 318], [285, 121, 321, 225, 293], [323, 433, 61, 354, 90], [433, 12, 5, 420, 242], [117, 118, 187, 396, 141], [335, 286, 18, 3

In [45]:
import json
output_dir = f"../icl/few_data_split/icl_for_dev"
with open(f"{output_dir}/{ICL_METHOD}.txt", "w") as jsonl_file:
    for icl in icls:
        json.dump(icl, jsonl_file)
        jsonl_file.write("\n")